In [136]:
import numpy as np
import os
import pandas as pd

import arcpy
from arcpy.sa import *

# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

arcpy.env.overwriteOutput = True # make sure overwrite files is on

pd.options.display.float_format = '{:.3f}'.format  # MAKE DAtaframe not display scientific notation

homedir = os.getcwd()
workspace = os.path.join(homedir, 'Workspace')


### Compile Rainfall amounts for each watershed that has a stream gauge

In [137]:
measured_watersheds = os.path.join(homedir, 'Data', 'Measured_watersheds.shp')


mo_list = ["01", "02", "03","04", "05", "06","07", "08", "09","10", "11", "12"]
mo_days = [31, 28 , 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
precp_dict = {}

for idx, i in enumerate(mo_list):

# run the zonal stats on the raster for each watershed and export as a table ZonalStatisticsAsTable (in_zone_data, zone_field, in_value_raster, out_table, {ignore_nodata}, {statistics_type})
    temp_raster = os.path.join(homedir, 'Data', 'Gridded_rain', 'PRISM_ppt_tutuila_30yr_normal_80mM1_{}_asc.asc'.format(i))


    # using measured watershed areas, calculate the total sum amount of water that precipitates in a given month in each wathershed area, save to table 
    outZSaT = ZonalStatisticsAsTable(measured_watersheds,  "Zone_ID",  temp_raster,  os.path.join(workspace, 'dumb_table.dbf'), "DATA")
    # make the table useful as opposed to arc dumb format
    arcpy.TableToTable_conversion(outZSaT, workspace, 'Zonal_stats_precip_mon_.csv')
    arcpy.Delete_management(os.path.join(workspace,'dumb_table.dbf'))  

    # each table needs some reformatting to get volumetric water amounts 
    data = pd.read_csv(os.path.join(workspace, 'Zonal_stats_precip_mon_.csv'))    # read into pandas dtaframe
    data['ave_precip_in_m'] = (data['MEAN']/100000)                                # data units are in mm*100, convert to m

    # need to read in the shapefile of measured watersheds to get the areas to calculate volumetirc amounts of water
    arr = arcpy.da.TableToNumPyArray(measured_watersheds, ("Zone_ID", 'area_m2'))   # read shapefile into an array
    arcistheworst = list(zip(*arr))                                                 # why would you make your arrays out of tuples? 
    dataset = pd.DataFrame({'Zone_ID':arcistheworst[0],'area_m2':arcistheworst[1]}) # convert dumb tuple array to pandas dataframe of areas

    data = data.merge(dataset, how='outer', on='Zone_ID')  # add areas to the dataframe
    data['precip_vol_m3pmonth'] = data['ave_precip_in_m']*data['area_m2']    # get it into volumne per month
    data['precip_vol_m3pd'] = data['precip_vol_m3pmonth']/mo_days[idx]   # get it into volumne per month

    
    pert_frame = data[['Zone_ID', 'precip_vol_m3pd']]
    precp_dict["month_{}".format(i)] = pert_frame

### Format the rainfall and runoff data into runoff to rainfall ratios for measured watersheds
- raw monthly consolidated runoff from ASPA-UH_Integrated_Modeling_Framework/ASPA-UH_Stream_REPO/Run/UH_ASPA_Streamflow_process_w_USGS.ipynb 
- raw precip data from precp_dict in this notebook

In [207]:
Raw_RO_data_path = os.path.join("..", "..", "..", "ASPA-UH_Stream_REPO", "workspace", "Raw_RO_data_Measured.csv")

# format the precipitation data into zones as columns
precip_frame = pd.DataFrame(columns= [47, 48, 63, 73, 85, 118, 125, 133, 163, 168, 236, 241, 272, 307, 342])

for i in precp_dict:
    tempframe = precp_dict[i].set_index('Zone_ID').transpose()
    tempframe['month'] = i
    tempframe = tempframe.set_index('month')
    precip_frame = precip_frame.append(tempframe)
    
# format the streamflow datasheet 
RO_raw = pd.read_csv(Raw_RO_data_path)              
RO_raw = RO_raw.set_index('Zone_ID')
del RO_raw['Site']
RO_raw = RO_raw.sort_index()

# pull it apart and atranspose it
RO_frame  = pd.DataFrame(columns = [], data = [])
for row in RO_raw.iterrows():
    temp_frame = pd.DataFrame(columns = [row[0]], data = row[1])
    RO_frame = RO_frame.join(temp_frame, on=None, how='outer') 
    
# make the index exactly match the precip dataset for the division
RO_frame = RO_frame.reset_index()
RO_frame['month_idx'] = RO_frame['index'].apply(lambda x: str(x.split("_")[0])+"_"+str(x.split("_")[1]))
RO_frame = RO_frame.set_index('month_idx')
del RO_frame['index']

#calculate the runoff to rainfall ratios
RO_to_RF_Ratios = RO_frame.divide(precip_frame, axis='rows')


"""Due to the short dataset, some of these ratios are outside the boundaries of what is reasonable. Reasonable is here defined by the recorded runoff to rainfall rations by Perault 2010, using USGS data, 
Therefore if the  calculated rations are <16% or are >54% then set them equal to those from Izuka and perault which were = 0.307 for west and 0.255 for east and the average of this is 0.281 """
for y in ydim:
    for x in xdim:
        if RO_to_RF_Ratios.iloc[y,x] > 0.54:
            RO_to_RF_Ratios.iloc[y,x] = 0.281
            
        if RO_to_RF_Ratios.iloc[y,x] < 0.16:
            RO_to_RF_Ratios.iloc[y,x] = 0.281
            
        if ~np.isfinite(RO_to_RF_Ratios.iloc[y,x]):
            RO_to_RF_Ratios.iloc[y,x] = 0.281



### This cell interpolates RO:RF ratios from measured watersheds to ungauged watersheds based on proximity and creates an output dataset for SWB2
Proximity was manually determined based on manual assignemt from examination of a map. 

Alos note that Tafuna - Leone RO:RF ratios are different based on the MFR effect (see documentation)

In [315]:
All_RoRf_ratios = pd.DataFrame(index = RO_to_RF_Ratios.index)   # create empty dataframe
All_RoRf_ratios["1"] =RO_to_RF_Ratios[[118, 133]].mean(axis=1)
All_RoRf_ratios["2"] =RO_to_RF_Ratios[[236, 133]].mean(axis=1)
All_RoRf_ratios["3"] =RO_to_RF_Ratios[[307, 342]].mean(axis=1)
All_RoRf_ratios["4"] =RO_to_RF_Ratios[[307]].mean(axis=1)
All_RoRf_ratios["5"] =RO_to_RF_Ratios[[236, 307]].mean(axis=1)
All_RoRf_ratios["6"] =RO_to_RF_Ratios[[272]].mean(axis=1)
All_RoRf_ratios["7"] =RO_to_RF_Ratios[[307]].mean(axis=1)
All_RoRf_ratios["8"] =RO_to_RF_Ratios[[342]].mean(axis=1)
All_RoRf_ratios["9"] =RO_to_RF_Ratios[[272, 342]].mean(axis=1)
All_RoRf_ratios["10"] =RO_to_RF_Ratios[[133, 241]].mean(axis=1)
All_RoRf_ratios["11"] =RO_to_RF_Ratios[[47, 63]].mean(axis=1)
All_RoRf_ratios["12"] =RO_to_RF_Ratios[[73, 85, 47]].mean(axis=1)
All_RoRf_ratios["13"] =RO_to_RF_Ratios[[47]].mean(axis=1)
All_RoRf_ratios["14"] =RO_to_RF_Ratios[[47, 48]].mean(axis=1)
All_RoRf_ratios["15"] = 0.065    # note this is tafuna Leone with special runoff properties
All_RoRf_ratios["16"] =RO_to_RF_Ratios[[47, 48]].mean(axis=1)
All_RoRf_ratios["17"] =RO_to_RF_Ratios[[47, 48]].mean(axis=1)
All_RoRf_ratios["18"] = 0.065
All_RoRf_ratios["19"] = 0.065
All_RoRf_ratios["20"] =RO_to_RF_Ratios[47]
All_RoRf_ratios["21"] =RO_to_RF_Ratios[63]
All_RoRf_ratios["22"] =RO_to_RF_Ratios[73]
All_RoRf_ratios["23"] =RO_to_RF_Ratios[85]
All_RoRf_ratios["24"] =RO_to_RF_Ratios[118]
All_RoRf_ratios["25"] =RO_to_RF_Ratios[125]
All_RoRf_ratios["26"] =RO_to_RF_Ratios[133]
All_RoRf_ratios["27"] =RO_to_RF_Ratios[163]
All_RoRf_ratios["28"] =RO_to_RF_Ratios[168]
All_RoRf_ratios["29"] =RO_to_RF_Ratios[236]
All_RoRf_ratios["30"] =RO_to_RF_Ratios[241]
All_RoRf_ratios["31"] =RO_to_RF_Ratios[272]
All_RoRf_ratios["32"] =RO_to_RF_Ratios[307]
All_RoRf_ratios["33"] =RO_to_RF_Ratios[342]

# create a date column for the desired years of the simualtion 
date_col = []
for year in range(2000,2012):
    for month in range(1,13):
        Date = "{}/1/{}".format(month, year)
        date_col.append(Date)
date_col_frame = pd.DataFrame(index = date_col)

# make a repeating dataframe of the RoRF values
two =    pd.concat([All_RoRf_ratios, All_RoRf_ratios])
four =   pd.concat([two, two])
eight =  pd.concat([four, four])
twelve = pd.concat([eight, four])
twelve["Date"] =  date_col          # merge on date column to the data

# print out a copy of the RO-RF data to be ingested into SWB2
RO_Rf_ratios_Dynamic_monthly_2000_2011 = twelve.set_index('Date')
RO_Rf_ratios_Dynamic_monthly_2000_2011.to_csv(os.path.join(".", 'RO_Rf_ratios_Dynamic_monthly_2000_2011.txt'), header=True, index=True, sep='\t', mode='a', float_format = "%.5f")